In [2]:
def getOriginalClass(valueMain,valueSec):
  valueMain = str(valueMain)
  valueSec = str(valueSec)

  dicClass = {
    '1':'Water',
    '2':'Ice/snow',
    '3':'Developed',
    '4':'Soil',
    '5':'Rock',
    '6':'Beach/sand',
    '7':'Deciduous',
    '8':'Evergreen',
    '9':'Mixed',
    '10':'Shrub',
    '11':'Grassland',
    '12':'Agriculture',
    '13':'Moss/lichen'
  }
  dicClassSec = {
      '4':'Barren/sparsely vegetated',
      '5':'Trees',
      '7':'Herbaceous'
  }
  if valueMain == '0':
    cls = dicClassSec[valueSec]
  else:
    cls = dicClass[valueMain]

  return cls

def getClassGPW(value):
  dicClass = {
    'Water':['Other',0],
    'Ice/snow':['Other',0],
    'Developed':['Other',0],
    'Soil':['Other',1],
    'Rock':['Other',1],
    'Beach/sand':['Other',1],
    'Deciduous':['Other',1],
    'Evergreen':['Other',1],
    'Mixed':['Other',1],
    'Shrub':['Natural and Semi-natural grass',2],
    'Grassland':['Seeded grass',3],
    'Agriculture':['Crops and other related agricultural practices',2],
    'Moss/lichen':['Natural and Semi-natural grass',1],
    'Barren/sparsely vegetated':['Other',1],
    'Trees':['Other',1],
    'Herbaceous':['not classify',0]
  }
  return dicClass[value]

In [3]:
import geopandas as gpd

In [4]:
src = '../datasets/12_Glance/bu_glance_training_dataV1.parquet'
gdf = gpd.read_parquet(src)
gdf

,Lat,Lon,Start_Year,End_Year,Dataset_Code,Continent_Code,LC_Confidence,Segment_Type,Leaf_Type,Impervious_Percent,...,Tree_Location,Veg_Modifier,Level1_Ecoregion,Level2_Ecoregion,Glance_Class_ID_level1,Glance_Class_ID_level2,Change,ID,Glance_ID,geometry
0,1.414980,-71.083640,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,5,0,False,0,02_7_00_5_0_1985_2018,POINT (-71.08364 1.41498)
1,1.972296,-70.944320,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,5,0,False,1,02_7_00_5_1_1985_2018,POINT (-70.94432 1.97230)
2,2.097071,-73.839766,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,7,11,False,2,02_7_00_5_2_1985_2018,POINT (-73.83977 2.09707)
3,1.205852,-71.773552,1986,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,1,1,False,3,02_7_00_5_3_1986_2018,POINT (-71.77355 1.20585)
4,1.205852,-71.775171,1986,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,1,1,False,4,02_7_00_5_4_1986_2018,POINT (-71.77517 1.20585)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874990,46.911576,10.391939,2018,2018,902,4,NaN,NaN,NaN,NaN,...,NaN,None,1,1_00,7,11,False,1641504,04_1_00_902_1641504_2018_2018,POINT (10.39194 46.91158)
1874991,46.410123,9.479237,2018,2018,902,4,NaN,NaN,NaN,NaN,...,NaN,None,1,1_00,7,11,False,1641505,04_1_00_902_1641505_2018_2018,POINT (9.47924 46.41012)
1874992,46.421422,9.685952,2018,2018,902,4,NaN,NaN,NaN,NaN,...,NaN,None,1,1_00,7,11,False,1641506,04_1_00_902_1641506_2018_2018,POINT (9.68595 46.42142)
1874993,46.890716,9.951898,2018,2018,902,4,NaN,NaN,NaN,NaN,...,NaN,None,1,1_00,7,11,False,1641507,04_1_00_902_1641507_2018_2018,POINT (9.95190 46.89072)


In [5]:
gdfDev = gdf.copy()
gdfDev = gdfDev[['End_Year','Glance_Class_ID_level1','Glance_Class_ID_level2','geometry']]
gdfDev['dataset_name'] = 'GLanCE'
gdfDev['original_lulc_class'] = gdfDev.apply(lambda x: getOriginalClass(x.Glance_Class_ID_level2,x.Glance_Class_ID_level1),axis=1)
gdfDev['gpw_lulc_class'] = gdfDev['original_lulc_class'].apply(lambda x: getClassGPW(x)[0])
gdfDev['sample_weight'] = gdfDev['original_lulc_class'].apply(lambda x: getClassGPW(x)[1])
gdfDev['observation'] = ''
gdfDev.rename(columns={
    'End_Year':'reference_year'
},inplace=True)

gdfDev

,reference_year,Glance_Class_ID_level1,Glance_Class_ID_level2,geometry,dataset_name,original_lulc_class,gpw_lulc_class,sample_weight,observation
0,2018,5,0,POINT (-71.08364 1.41498),GLanCE,Trees,Other,1,
1,2018,5,0,POINT (-70.94432 1.97230),GLanCE,Trees,Other,1,
2,2018,7,11,POINT (-73.83977 2.09707),GLanCE,Grassland,Seeded grass,3,
3,2018,1,1,POINT (-71.77355 1.20585),GLanCE,Water,Other,0,
4,2018,1,1,POINT (-71.77517 1.20585),GLanCE,Water,Other,0,
...,...,...,...,...,...,...,...,...,...
1874990,2018,7,11,POINT (10.39194 46.91158),GLanCE,Grassland,Seeded grass,3,
1874991,2018,7,11,POINT (9.47924 46.41012),GLanCE,Grassland,Seeded grass,3,
1874992,2018,7,11,POINT (9.68595 46.42142),GLanCE,Grassland,Seeded grass,3,
1874993,2018,7,11,POINT (9.95190 46.89072),GLanCE,Grassland,Seeded grass,3,


In [6]:
gdfDev = gdfDev[['dataset_name', 'reference_year', 'original_lulc_class', 'gpw_lulc_class', 'sample_weight','observation','geometry']]
gdfDev.head(10)

,dataset_name,reference_year,original_lulc_class,gpw_lulc_class,sample_weight,observation,geometry
0,GLanCE,2018,Trees,Other,1,,POINT (-71.08364 1.41498)
1,GLanCE,2018,Trees,Other,1,,POINT (-70.94432 1.97230)
2,GLanCE,2018,Grassland,Seeded grass,3,,POINT (-73.83977 2.09707)
3,GLanCE,2018,Water,Other,0,,POINT (-71.77355 1.20585)
4,GLanCE,2018,Water,Other,0,,POINT (-71.77517 1.20585)
5,GLanCE,2018,Water,Other,0,,POINT (-71.77543 1.20612)
6,GLanCE,2018,Water,Other,0,,POINT (-74.78219 1.22121)
7,GLanCE,2018,Water,Other,0,,POINT (-74.78138 1.22391)
8,GLanCE,2018,Water,Other,0,,POINT (-71.71642 1.23307)
9,GLanCE,2018,Water,Other,0,,POINT (-74.82207 1.27538)


In [7]:
srcout = '/media/bernard/Data/LAPIG/Doutorado/Global_Pasture_Watch/datasets/harmonized_databases/12_db_glance.parquet'
gdfDev.to_parquet(srcout)